https://www.kaggle.com/code/purvanshsingh/keras-tuner-with-dataimagegenerator/notebook
https://codelabs.developers.google.com/codelabs/keras-flowers-squeezenet#3

In [ ]:
!pip install --upgrade --no-cache-dir gdown
!gdown 1Srhsx2y5Yhuia7FPVqhdhpIAJNE4Kv0W
!pip install -U keras-tuner
!unzip MIT_small_train_1.zip
!pip install optuna
!pip install -U kaleido

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: __MACOSX/MIT_small_train_1/test/forest/._nat1088.jpg  
  inflating: MIT_small_train_1/test/forest/land870.jpg  
  inflating: __MACOSX/MIT_small_train_1/test/forest/._land870.jpg  
  inflating: MIT_small_train_1/test/forest/cdmc338.jpg  
  inflating: __MACOSX/MIT_small_train_1/test/forest/._cdmc338.jpg  
  inflating: MIT_small_train_1/test/forest/land858.jpg  
  inflating: __MACOSX/MIT_small_train_1/test/forest/._land858.jpg  
  inflating: MIT_small_train_1/test/forest/nat470.jpg  
  inflating: __MACOSX/MIT_small_train_1/test/forest/._nat470.jpg  
  inflating: MIT_small_train_1/test/forest/land871.jpg  
  inflating: __MACOSX/MIT_small_train_1/test/forest/._land871.jpg  
  inflating: MIT_small_train_1/test/forest/for28.jpg  
  inflating: __MACOSX/MIT_small_train_1/test/forest/._for28.jpg  
  inflating: MIT_small_train_1/test/forest/cdmc271.jpg  
  inflating: __MACOSX/MIT_small_train_1/test/forest/.

In [ ]:
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import optuna
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten, Reshape
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from tensorflow.keras.activations import softmax, softplus, softsign, relu, tanh, sigmoid, hard_sigmoid, linear
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [ ]:
train_data_dir = "/content/MIT_small_train_1/train"
test_data_dir = "/content/MIT_small_train_1/test"
MODEL_FNAME = "model_trained.h5"
folderHyper = "findHypers/"

img_width = 32
img_height= 32
validation_samples=2288

In [ ]:
block = [True, False]
batch_sizes = [8, 16, 32, 64, 128]
number_of_epochs = [10, 50, 100]
optimizersIndex = ["SGD", "RMSprop", "Adagrad", "Adadelta", "Adam", "Adamax", "Nadam"]
learning_rates = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
momentums = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
activationsIndex = ["softmax", "softplus", "softsign", "relu", "tanh", 
                    "sigmoid", "hard_sigmoid", "Linear"]
activations = {"softmax": softmax, "softplus": softplus, 
               "softsign": softsign, "relu": relu, "tanh": tanh, 
               "sigmoid": sigmoid, "hard_sigmoid": hard_sigmoid, 
               "Linear": linear}
data_augmentations = {"width_shift_range": [0.1, 0], "height_shift_range": [0.1, 0], 
                      "horizontal_flip": [True, False], "vertical_flip": [True, False],
                      "rotation_range": [10,0], "brightness_range": [True, False],
                      "zoom_range": [0.1, 0], "shear_range": [10, 0]}

In [ ]:
bnmomemtum=0.9
def fire(x, squeeze, expand, activation):
  y  = tf.keras.layers.Conv2D(filters=squeeze, kernel_size=1, activation= activation, padding='same')(x)
  y = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y)
  y1 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=1, activation= activation, padding='same')(y)
  y1 = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y1)
  y3 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=3, activation= activation, padding='same')(y)
  y3 = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y3)
  return tf.keras.layers.concatenate([y1, y3])

def fire_module(squeeze, expand, activation):
  return lambda x: fire(x, squeeze, expand, activation)

def model(trial):

  # Choose hyperparameters
  first_block = trial.suggest_categorical("first_block", block)
  second_block = trial.suggest_categorical("second_block", block)
  third_block = trial.suggest_categorical("third_block", block)
  fourth_block = trial.suggest_categorical("fourth_block", block)
  batch_size = trial.suggest_categorical("batch_size", batch_sizes)
  number_of_epoch = trial.suggest_categorical("epochs", number_of_epochs)
  optimizerIndex = trial.suggest_categorical("optimizer", optimizersIndex)
  learning_rate = trial.suggest_categorical("learning_rate", learning_rates)
  activationIndex = trial.suggest_categorical("activation", activationsIndex)
  activation = activations[activationIndex]

  width_shift_range = trial.suggest_categorical("width_shift_range", data_augmentations["width_shift_range"])
  height_shift_range = trial.suggest_categorical("height_shift_range", data_augmentations["height_shift_range"])
  horizontal_flip = trial.suggest_categorical("horizontal_flip", data_augmentations["horizontal_flip"])
  vertical_flip = trial.suggest_categorical("vertical_flip", data_augmentations["vertical_flip"])
  rotation_range = trial.suggest_categorical("rotation_range", data_augmentations["rotation_range"])
  brightness_range = trial.suggest_categorical("brightness_range", data_augmentations["brightness_range"])
  if brightness_range:
      brightness_range = [0.8,1.2]
  else:
      brightness_range = [1.,1.]
  zoom_range = trial.suggest_categorical("zoom_range", data_augmentations["zoom_range"])
  shear_range = trial.suggest_categorical("shear_range", data_augmentations["shear_range"])

  
  x = tf.keras.layers.Input(shape=[img_width, img_height, 3]) 
  x = tf.keras.layers.Reshape((img_width, img_height, 3))(x)

  y = tf.keras.layers.Conv2D(kernel_size=3, filters=32, padding='same', use_bias=True, activation= activation)(x)

  y = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y)

  if first_block == True:
    y = fire_module(24, 48, activation)(y)
    y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)

  if second_block == True:
    y = fire_module(48, 96, activation)(y)
    y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)

  if third_block == True:
    y = fire_module(64, 128, activation)(y)
    y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)

  if fourth_block == True:
    y = fire_module(48, 96, activation)(y)
    y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)

  y = fire_module(24, 48, activation)(y)
  y = tf.keras.layers.GlobalAveragePooling2D()(y)

  y = tf.keras.layers.Dense(8, activation= activation)(y)

  model = tf.keras.Model(x, y)

  model.compile(
    optimizer= optimizerIndex,
    loss= 'categorical_crossentropy',
    metrics=['accuracy'])
  
  model.summary()

  datagenTrain = ImageDataGenerator(
      rescale = 1.0/255,
      featurewise_center=False,
      samplewise_center=False,
      featurewise_std_normalization=False,
      samplewise_std_normalization=False,
      rotation_range=rotation_range,
      width_shift_range=width_shift_range,
      height_shift_range=height_shift_range,
      shear_range=shear_range,
      zoom_range=zoom_range,
      brightness_range= brightness_range,
      channel_shift_range=0.,
      fill_mode='nearest',
      cval=0.,
      horizontal_flip=horizontal_flip,
      vertical_flip=vertical_flip,
      )
  
  datagenTest = ImageDataGenerator(
      rescale = 1.0/255,
      featurewise_center=False,
      samplewise_center=False,
      featurewise_std_normalization=False,
      samplewise_std_normalization=False,
      rotation_range=0.,
      width_shift_range=0.,
      height_shift_range=0.,
      shear_range=0.,
      zoom_range=0.,
      channel_shift_range=0.,
      fill_mode='nearest',
      cval=0.,
      horizontal_flip=False,
      vertical_flip=False,
      )
  
  train_generator = datagenTrain.flow_from_directory(train_data_dir,
          target_size=(img_width, img_height),
          batch_size=batch_size,
          class_mode='categorical')
  
  test_generator = datagenTest.flow_from_directory(test_data_dir,
          target_size=(img_width, img_height),
          batch_size=batch_size,
          class_mode='categorical')

  # To save the best model
  checkpointer = ModelCheckpoint(filepath=MODEL_FNAME, verbose=1, save_best_only=True, 
                                  monitor='val_accuracy')
  
  history=model.fit(train_generator,
          steps_per_epoch=(int(np.ceil(400/batch_size))),
          epochs=number_of_epoch,
          validation_data=test_generator,
          validation_steps= (int(np.ceil(validation_samples/batch_size))), callbacks=[checkpointer])
  
  result = model.evaluate(test_generator)
  
  newFolder = folderHyper + "trial" + str(trial.number) + "/"
  if not os.path.exists(newFolder):
      os.makedirs(newFolder)
  model.save(newFolder + MODEL_FNAME)

  # Save results
  file  = open(newFolder + "results.txt", "w")
  file.write(f"Score: {result}\n")
  file.write(f"Batch size: {batch_size}\n")
  file.write(f"Epochs: {number_of_epoch}\n")
  file.write(f"Optimizer: {optimizerIndex}\n")
  file.write(f"Lr: {learning_rate}\n")
  if optimizerIndex == "SGD" or optimizerIndex == "RMSprop":
      file.write(f"Momentum: {bnmomemtum}\n")
  file.write(f"Activation: {activationIndex}\n")
  file.write(f"first_block: {first_block}\n")
  file.write(f"second_block: {second_block}\n")
  file.write(f"third_block: {third_block}\n")
  file.write(f"fourth_block: {fourth_block}\n")
  file.write("DATA AUGMENTATION: \n")
  file.write(f"width_shift_range: {width_shift_range}\n")
  file.write(f"height_shift_range: {height_shift_range}\n")
  file.write(f"horizontal_flip: {horizontal_flip}\n")
  file.write(f"vertical_flip: {vertical_flip}\n")
  file.write(f"rotation_range: {rotation_range}\n")
  file.write(f"brightness_range: {brightness_range}\n")
  file.write(f"zoom_range: {zoom_range}\n")
  file.write(f"shear_range: {shear_range}\n")
  file.close()
  
  fig1, ax1 = plt.subplots()
  ax1.plot(history.history['accuracy'])
  ax1.plot(history.history['val_accuracy'])
  ax1.set_title('model accuracy')
  ax1.set_ylabel('accuracy')
  ax1.set_xlabel('epoch')
  ax1.legend(['train', 'validation'], loc='upper left')
  fig1.savefig(newFolder + 'accuracy.jpg')
  plt.close(fig1)
    # summarize history for loss
  fig1, ax1 = plt.subplots()
  ax1.plot(history.history['loss'])
  ax1.plot(history.history['val_loss'])
  ax1.set_title('model loss')
  ax1.set_ylabel('loss')
  ax1.set_xlabel('epoch')
  ax1.legend(['train', 'validation'], loc='upper left')
  fig1.savefig(newFolder + 'loss.jpg')
    
  plot_model(model, to_file=newFolder + 'mymodel.png', show_shapes=True, show_layer_names=True)

  return result[1]

# Create Study object
study = optuna.create_study(direction="maximize")
# Optimize the study
study.optimize(model, n_trials=5) # Use more 
# Print the result
best_params = study.best_params
best_score = study.best_value
print(f"Best score: {best_score}\n")
print(f"Optimized parameters: {best_params}\n")

[I 2023-02-04 12:17:01,481] A new study created in memory with name: no-name-e2114940-ad33-4753-b760-a52e7ac2adcc


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 32)   896         ['input_2[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 32)  128         ['conv2d[1][0]']                 
 alization)                                                                                       
                                                                                                  
 conv2d_1 (Conv2D)              (None, 32, 32, 48)   1584        ['batch_normalization[1][0]']

[I 2023-02-04 12:27:12,953] Trial 0 finished with value: 0.5336538553237915 and parameters: {'first_block': False, 'second_block': True, 'third_block': False, 'fourth_block': True, 'batch_size': 128, 'epochs': 100, 'optimizer': 'RMSprop', 'learning_rate': 0.1, 'activation': 'hard_sigmoid', 'width_shift_range': 0.1, 'height_shift_range': 0, 'horizontal_flip': False, 'vertical_flip': False, 'rotation_range': 10, 'brightness_range': True, 'zoom_range': 0.1, 'shear_range': 0}. Best is trial 0 with value: 0.5336538553237915.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_10 (Conv2D)             (None, 32, 32, 32)   896         ['input_4[0][0]']                
                                                                                                  
 batch_normalization_10 (BatchN  (None, 32, 32, 32)  128         ['conv2d_10[1][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_11 (Conv2D)             (None, 32, 32, 24)   792         ['batch_normalization_10[1]

[I 2023-02-04 12:37:14,318] Trial 1 finished with value: 0.09571678191423416 and parameters: {'first_block': True, 'second_block': True, 'third_block': False, 'fourth_block': False, 'batch_size': 16, 'epochs': 100, 'optimizer': 'Nadam', 'learning_rate': 0.01, 'activation': 'tanh', 'width_shift_range': 0, 'height_shift_range': 0, 'horizontal_flip': False, 'vertical_flip': False, 'rotation_range': 0, 'brightness_range': False, 'zoom_range': 0.1, 'shear_range': 0}. Best is trial 0 with value: 0.5336538553237915.


...

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_33 (Conv2D)             (None, 32, 32, 32)   896         ['input_8[0][0]']                
                                                                                                  
 batch_normalization_33 (BatchN  (None, 32, 32, 32)  128         ['conv2d_33[1][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_34 (Conv2D)             (None, 32, 32, 24)   792         ['batch_normalization_33[1]

[I 2023-02-04 12:42:54,283] Trial 3 finished with value: 0.3841783106327057 and parameters: {'first_block': False, 'second_block': False, 'third_block': False, 'fourth_block': False, 'batch_size': 32, 'epochs': 50, 'optimizer': 'SGD', 'learning_rate': 0.001, 'activation': 'softplus', 'width_shift_range': 0.1, 'height_shift_range': 0, 'horizontal_flip': False, 'vertical_flip': False, 'rotation_range': 0, 'brightness_range': True, 'zoom_range': 0.1, 'shear_range': 0}. Best is trial 0 with value: 0.5336538553237915.


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_37 (Conv2D)             (None, 32, 32, 32)   896         ['input_10[0][0]']               
                                                                                                  
 batch_normalization_37 (BatchN  (None, 32, 32, 32)  128         ['conv2d_37[1][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_38 (Conv2D)             (None, 32, 32, 48)   1584        ['batch_normalization_37[1]

[I 2023-02-04 12:47:28,716] Trial 4 finished with value: 0.7460664510726929 and parameters: {'first_block': False, 'second_block': True, 'third_block': True, 'fourth_block': True, 'batch_size': 64, 'epochs': 50, 'optimizer': 'Adam', 'learning_rate': 0.1, 'activation': 'sigmoid', 'width_shift_range': 0, 'height_shift_range': 0, 'horizontal_flip': False, 'vertical_flip': False, 'rotation_range': 10, 'brightness_range': True, 'zoom_range': 0, 'shear_range': 0}. Best is trial 4 with value: 0.7460664510726929.


Best score: 0.7460664510726929

Optimized parameters: {'first_block': False, 'second_block': True, 'third_block': True, 'fourth_block': True, 'batch_size': 64, 'epochs': 50, 'optimizer': 'Adam', 'learning_rate': 0.1, 'activation': 'sigmoid', 'width_shift_range': 0, 'height_shift_range': 0, 'horizontal_flip': False, 'vertical_flip': False, 'rotation_range': 10, 'brightness_range': True, 'zoom_range': 0, 'shear_range': 0}



In [ ]:
# Create Study object
study = optuna.create_study(direction="maximize")
# Optimize the study
study.optimize(model, n_trials=10) # Use more 
# Print the result
best_params = study.best_params
best_score = study.best_value
print(f"Best score: {best_score}\n")
print(f"Optimized parameters: {best_params}\n")

[I 2023-02-04 12:47:28,863] A new study created in memory with name: no-name-051e6743-2d6c-435a-9c45-8d325532d0bb


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_50 (Conv2D)             (None, 32, 32, 32)   896         ['input_12[0][0]']               
                                                                                                  
 batch_normalization_50 (BatchN  (None, 32, 32, 32)  128         ['conv2d_50[1][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_51 (Conv2D)             (None, 32, 32, 64)   2112        ['batch_normalization_50[1]

[I 2023-02-04 12:51:49,252] Trial 0 finished with value: 0.5581293702125549 and parameters: {'first_block': False, 'second_block': False, 'third_block': True, 'fourth_block': True, 'batch_size': 128, 'epochs': 50, 'optimizer': 'SGD', 'learning_rate': 0.3, 'activation': 'softplus', 'width_shift_range': 0.1, 'height_shift_range': 0.1, 'horizontal_flip': True, 'vertical_flip': False, 'rotation_range': 0, 'brightness_range': True, 'zoom_range': 0.1, 'shear_range': 0}. Best is trial 0 with value: 0.5581293702125549.


Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_60 (Conv2D)             (None, 32, 32, 32)   896         ['input_14[0][0]']               
                                                                                                  
 batch_normalization_60 (BatchN  (None, 32, 32, 32)  128         ['conv2d_60[1][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_61 (Conv2D)             (None, 32, 32, 48)   1584        ['batch_normalization_60[1]

[I 2023-02-04 12:52:51,201] Trial 1 finished with value: 0.11931817978620529 and parameters: {'first_block': False, 'second_block': True, 'third_block': False, 'fourth_block': False, 'batch_size': 128, 'epochs': 10, 'optimizer': 'RMSprop', 'learning_rate': 0.1, 'activation': 'softplus', 'width_shift_range': 0.1, 'height_shift_range': 0, 'horizontal_flip': False, 'vertical_flip': True, 'rotation_range': 0, 'brightness_range': True, 'zoom_range': 0, 'shear_range': 0}. Best is trial 0 with value: 0.5581293702125549.


...

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_28 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_124 (Conv2D)            (None, 32, 32, 32)   896         ['input_28[0][0]']               
                                                                                                  
 batch_normalization_124 (Batch  (None, 32, 32, 32)  128         ['conv2d_124[1][0]']             
 Normalization)                                                                                   
                                                                                                  
 conv2d_125 (Conv2D)            (None, 32, 32, 24)   792         ['batch_normalization_124[

[I 2023-02-04 13:28:50,496] Trial 8 finished with value: 0.15690559148788452 and parameters: {'first_block': True, 'second_block': False, 'third_block': False, 'fourth_block': False, 'batch_size': 64, 'epochs': 100, 'optimizer': 'Nadam', 'learning_rate': 0.2, 'activation': 'softsign', 'width_shift_range': 0, 'height_shift_range': 0.1, 'horizontal_flip': False, 'vertical_flip': False, 'rotation_range': 0, 'brightness_range': True, 'zoom_range': 0.1, 'shear_range': 10}. Best is trial 3 with value: 0.629807710647583.


Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_30 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_131 (Conv2D)            (None, 32, 32, 32)   896         ['input_30[0][0]']               
                                                                                                  
 batch_normalization_131 (Batch  (None, 32, 32, 32)  128         ['conv2d_131[1][0]']             
 Normalization)                                                                                   
                                                                                                  
 conv2d_132 (Conv2D)            (None, 32, 32, 24)   792         ['batch_normalization_131[

[I 2023-02-04 13:29:47,243] Trial 9 finished with value: 0.2937062978744507 and parameters: {'first_block': False, 'second_block': False, 'third_block': False, 'fourth_block': False, 'batch_size': 64, 'epochs': 10, 'optimizer': 'SGD', 'learning_rate': 0.001, 'activation': 'sigmoid', 'width_shift_range': 0, 'height_shift_range': 0.1, 'horizontal_flip': True, 'vertical_flip': True, 'rotation_range': 10, 'brightness_range': True, 'zoom_range': 0.1, 'shear_range': 10}. Best is trial 3 with value: 0.629807710647583.


Best score: 0.629807710647583

Optimized parameters: {'first_block': False, 'second_block': True, 'third_block': False, 'fourth_block': True, 'batch_size': 32, 'epochs': 100, 'optimizer': 'Nadam', 'learning_rate': 0.1, 'activation': 'sigmoid', 'width_shift_range': 0.1, 'height_shift_range': 0.1, 'horizontal_flip': True, 'vertical_flip': False, 'rotation_range': 10, 'brightness_range': True, 'zoom_range': 0.1, 'shear_range': 10}



In [ ]:
# plots
fig = optuna.visualization.plot_slice(study, params=['batch_size', 'epochs', 'optimizer', 
                                                     'learning_rate', "activation", "first_block"
                                                     ,"second_block", "third_block", "fourth_block"])
fig.write_image("plot_slice.png")
fig = optuna.visualization.plot_slice(study, params=['width_shift_range', 'height_shift_range', 'horizontal_flip', 
                                                     'vertical_flip', "rotation_range", "brightness_range",
                                                     "zoom_range", "shear_range"])
fig.write_image("plot_slice2.png")
fig = optuna.visualization.plot_contour(study, params=['batch_size', 'epochs', 'optimizer', 
                                                     'learning_rate', "activation", "first_block"
                                                     ,"second_block", "third_block", "fourth_block"])
fig.write_image("plot_contour.png")
fig = optuna.visualization.plot_contour(study, params=['width_shift_range', 'height_shift_range', 'horizontal_flip', 
                                                     'vertical_flip', "rotation_range", "brightness_range",
                                                     "zoom_range", "shear_range"])
fig.write_image("plot_contour2.png")
# Save results
file  = open("results.txt", "w")
file.write(f"Best score: {best_score}\n")
file.write(f"Optimized parameters: {best_params}\n")
file.close()

In [ ]:
!zip -r /content/sample_data.zip /content/findHypers